In [1]:
"""This file :
                1. Scrapes names and relevant links to player wiki from url below and stores this data in csv file.
                2. File from 1. """

#Import packages
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import requests

#1.Obtain list of all male tennis players and their wiki link
url="https://en.wikipedia.org/wiki/List_of_male_singles_tennis_players"
response = requests.get(url)
soup =  BeautifulSoup(response.text, 'html.parser')
a = soup.find('table',{'class':"wikitable"})
vikilist = []
namelist = []
for tr in a.find_all(class_='vcard'):
    x = tr.find("a")
    namelist.append(x.string)
    vikilist.append(x["href"])
d = {'Name':namelist,'Viki':vikilist}
df = pd.DataFrame(d)
df.to_excel('Names.xlsx')


In [2]:
%%time
j=0
players = BeautifulSoup()
for i in vikilist:
        url="https://en.wikipedia.org" + i
        response = requests.get(url)
        players[j] =  BeautifulSoup(response.text, 'html.parser')
        j+=1
#629

Wall time: 15min 45s


In [ ]:
players[4].table.find('th',string="Plays").next_sibling.text

In [ ]:
data = []
for i in range(len(vikilist)):
    try:
        x = players[i].table.find('span',class_="bday").text
        z = players[i].table.find('th',string="Plays").next_sibling.text
        y = i,x,z
        print(y)
    except AttributeError:
        print("Attribiute error")
    

In [ ]:
#        data.append(y)
#    except:
#         x =  "No data found"
#     except:
#             z = "No data found"
#y = players[i].find_all(class_='')print(vikilist[i],i,x,"h does not exist")

In [ ]:
data

In [ ]:
for i in range(len(vikilist[:100])):
        x = players[i].table.find_all('th')
        for x in x:
            if x.text == "Plays":
                print(x.attrts)
            else:
                print('x')

In [ ]:
z = players[3].table.find_all('th')
h = []
for z in z: 
    if z.text == "Plays":
        print(z.next_sibling.text)

In [ ]:
players[3].table.find_all(class_='infobox-data')[7].string

In [ ]:
x = players[1].find_all(class_='bday')
z = players[2].table.find('th',string="Plays").next_sibling.text

In [ ]:
x

In [ ]:
z

In [ ]:
players[3].table.find('span',class_="bday").string

In [ ]:
for tr in a.contents:
    print(tr)
    print('******************************')

In [ ]:
for a in soup.find_all('a', href=True):
    print("Found the URL:", a['href'])

In [ ]:
# from selenium import webdriver
# from bs4 import BeautifulSoup
# import pandas as pd
# from selenium.webdriver.common.by import By
# from selenium.common.exceptions import NoSuchElementException
# #Import player names file
# data=pd.read_csv('unique_player_names_to_scrape1.csv')

In [ ]:
# driver = webdriver.Chrome("C:/Users/pjagi/Downloads/chromedriver_win32/chromedriver.exe")
# date_of_birth=[] #List to store dob
# plays=[] #List to store hand
# backhand=[]#List to store backhand
# driver.get("https://en.wikipedia.org/")
# search_box = driver.find_element_by_xpath('/html/body/div[5]/div[1]/div[2]/div/div/form/div/input[1]')
# search_box.send_keys(data['name'][4]+' tennis')
# search_button = driver.find_element_by_xpath('/html/body/div[5]/div[1]/div[2]/div/div/form/div/input[4]')
# search_button.click()
# first_result_button = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[4]/div[3]/ul/li[1]/div[1]/a')
# first_result_button.click()
# dob_element = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table[1]/tbody/tr[5]/td')
# userid = dob_element.text
# userid

In [ ]:
# date_of_birth=[] #List to store dob
# plays=[] #List to store hand
# backhand=[]#List to store backhand
# #Loop over all names to pick up data
# for name in data['name'][:10]:
#     driver = webdriver.Chrome("C:/Users/pjagi/Downloads/chromedriver_win32/chromedriver.exe")
#     driver.get("https://en.wikipedia.org/")
#     search_box = driver.find_element_by_xpath('/html/body/div[5]/div[1]/div[2]/div/div/form/div/input[1]')
#     try:
#         search_box.send_keys(name)
#     search_button = driver.find_element_by_xpath('/html/body/div[5]/div[1]/div[2]/div/div/form/div/input[4]')
#     search_button.click()
#     first_result_button = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[4]/div[3]/ul/li[1]/div[1]/a')
#     first_result_button.click()
#     dob_element = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table[1]/tbody/tr[5]/td')
#     data['date_of_birth'] = dob_element.text